In [3]:
from urllib.parse import urlparse

import pandas as pd

In [4]:
file_path = "391211629_aurora-medical-center-manitowoc-county_standardcharges.xml"
url = "https://www.aurorahealthcare.org/assets/documents/billing-insurance/pricing-transparency/391211629_aurora-medical-center-manitowoc-county_standardcharges.xml"

In [5]:
from enum import Enum

class FileFormat(Enum):
    XML = "XML"
    CSV = "CSV"
    JSON = "JSON"
    XLSX = "XLSX"

class FileSubtype(Enum):
    AURORA = "AURORA"
    
class TypeRecognizer(object):
    def _looks_like_aurora_xml(self, file_path):
        try:
            df = pd.read_xml(file_path)
        except:
            return False
        
        columns = df.columns.to_list()
        
        if len(columns) > 10:
            check_colnames = ['Facility', 'Type', 'Chargecode_DRG_CPT', 'Description', 
                              'Rev', 'CPT', 'NDC', 'Self_Pay', 'Min', 'Max']
            for col_name in check_colnames:
                if not col_name in columns:
                    return False
            
            remaining_colnames = list(set(columns) - set(check_colnames))
            for col_name in remaining_colnames:
                if " " in col_name:
                    return False
                
                if not col_name.startswith("_"):
                    return False
                
                components = col_name.split("_")
                
                if len(components) < 3:
                    return False
                
                if len(components[1]) != 4 and components[-1] != 'Fee':
                    return False
                
                return True
        
        return False
    
    def recognize_format_and_subtype(self, file_path):
        file_format = None
        subtype = None
        
        if file_path.endswith(".xml") or file_path.endswith(".XML"):
            file_format = FileFormat.XML
            
            if self._looks_like_aurora_xml(file_path):
                subtype = FileSubtype.AURORA
        
        return file_format, subtype

In [6]:
class AbstractStandardChargesConverter(object):
    pass

In [7]:
class AuroraXMLConvert(object):
    pass

In [8]:
df_in = pd.read_xml(file_path)
# HACK: https://stackoverflow.com/a/50132405
df_in['Rev'] = df_in['Rev'].fillna(-1)
df_in['Rev'] = df_in['Rev'].astype(int)
df_in['Rev'] = df_in['Rev'].replace(-1, None)
df_in

,Facility,Type,Chargecode_DRG_CPT,Description,Rev,CPT,NDC,_1_1_23_Fee,_2023_Aetna_W,_2023_Aetna_PPO,...,_2023_Trilogy,_2023_UHC_Charter,_2023_UHC_Nexus,_2023_UHC_HMO,_2023_UHC_PPO,_2023_WPS_Arise,_2023_WPS_Statewide,Self_Pay,Min,Max
0,MANITOWOC,CHARGE,10000002,ROOM CHARGE MED SURG,121,None,NaN,"1,770.00","1,122.39","1,471.10",...,"1,335.14","1,332.39","1,332.39","1,426.36","1,497.42","1,235.34","1,396.45",973.50,"1,079.70","1,504.50"
1,MANITOWOC,CHARGE,10000003,ROOM CHARGE WOMEN'S HEALTH,122,None,NaN,"1,770.00",778.73,"1,483.26",...,920.84,963.36,963.36,"1,040.03","1,497.42",891.65,"1,059.77",973.50,755.66,"1,504.50"
2,MANITOWOC,CHARGE,10000004,ROOM CHARGE PEDIATRICS,123,None,NaN,"1,770.00",840.12,"1,483.26",...,"1,111.48","1,010.75","1,010.75","1,101.19","1,497.42",840.12,"1,012.80",973.50,812.02,"1,770.00"
3,MANITOWOC,CHARGE,10000005,ROOM CHARGE NURSERY LEVEL 1,171,None,NaN,"1,650.00",969.48,"1,382.70",...,"1,239.59","1,159.05","1,159.05","1,229.76","1,395.90","1,004.20","1,163.04",907.50,969.48,"1,402.51"
4,MANITOWOC,CHARGE,10000008,ROOM CHARGE ICU OR CCU,200,None,NaN,"4,070.00","2,417.19","3,410.66",...,"2,787.26","2,968.97","2,968.97","3,162.02","3,443.22","2,610.12","2,895.14","2,238.50","2,381.53","3,459.51"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3017,MANITOWOC,OP PROC*,66984,XCAPSL CTRC RMVL W/O ECP,None,NaN,NaN,"12,409.28","3,211.22","10,398.98",...,"4,117.54","2,950.00","2,950.00","3,214.00","10,498.25","3,401.22","4,089.22","6,825.10","2,741.57","12,331.01"
3018,MANITOWOC,OP PROC*,69436,CREATE EARDRUM OPENING,None,NaN,NaN,"12,013.22","9,509.09","10,067.08",...,"5,970.52","4,196.51","4,196.51","4,571.73","10,163.18","3,586.59","4,351.73","6,607.27","2,929.89","11,606.12"
3019,MANITOWOC,OP PROC*,G0105,COLORECTAL SCRN; HI RISK IND,None,NaN,NaN,"6,453.61","3,119.00","5,408.13",...,"2,163.00","2,360.00","2,360.00","2,571.00","5,459.76","1,790.00","2,172.00","3,549.49","1,060.00","6,453.61"
3020,MANITOWOC,OP PROC*,G0121,COLON CA SCRN NOT HI RSK IND,None,NaN,NaN,"6,476.92","3,119.00","5,427.66",...,"2,163.00","2,360.00","2,360.00","2,571.00","5,479.47","1,790.00","2,172.00","3,562.31","1,060.00","6,476.92"


In [9]:
recognizer = TypeRecognizer()
recognizer

In [10]:
recognizer.recognize_format_and_subtype(file_path)

(<FileFormat.XML: 'XML'>, <FileSubtype.AURORA: 'AURORA'>)